In [1]:
from scipy import stats as st
import pandas as pd

In [2]:
csv_path = "../dataset/preprocessed.csv"

In [3]:
data = pd.read_csv(csv_path)
data.describe()

,Age,Pack_Year,BMI,KILLIP_new,Gender_Female,Diabetes Mellitus,Hypertension,MR_Moderate,MR_Severe,VSR,LV ANEURYSM,KILLIP_new.1,TIMI,CardiacStatus_Presentation_Cardiogenic Shock,CardiacStatus_Presentation_Heart failure,mortality_30_days
count,1.700000e+03,1.700000e+03,1.700000e+03,1.700000e+03,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000
mean,1.630069e-16,7.105427e-17,1.320774e-15,6.687461e-17,0.194118,0.219412,0.314118,0.023529,0.012941,0.008824,0.012941,0.328235,8.521882,0.107647,0.006471,0.123529
std,1.000294e+00,1.000294e+00,1.000294e+00,1.000294e+00,0.395636,0.413970,0.464300,0.151622,0.113054,0.093546,0.113054,0.740983,9.407734,0.310025,0.080203,0.329141
min,-3.060544e+00,-7.197262e-01,-2.675582e+00,-4.431034e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.800000,0.000000,0.000000,0.000000
25%,-7.103434e-01,-7.197262e-01,-6.611399e-01,-4.431034e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.200000,0.000000,0.000000,0.000000
50%,4.507844e-02,-7.197262e-01,-5.867055e-02,-4.431034e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.400000,0.000000,0.000000,0.000000
75%,6.326287e-01,6.886417e-01,5.663490e-01,-4.431034e-01,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.400000,0.000000,0.000000,0.000000
max,3.486444e+00,3.249311e+00,4.738586e+00,2.256810e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,35.900000,1.000000,1.000000,1.000000


In [4]:
num = ['Age', 'Pack_Year', 'BMI', 'KILLIP_new']

In [5]:
for col in data.columns:
    data[col] = data[col].fillna(0)

In [6]:
target = "mortality_30_days"

In [7]:
labels = data[target].unique()
labels

array([0, 1])

In [8]:
test_stats = {}
p_vals = {}

alpha = 0.05

In [9]:
for col in data.columns:
    if(data[col].unique().shape[0]==1):
        continue

    if (col != target):
        
        counts = [data[data[target] == label][col] for label in labels]

        if col not in num:
            observed = pd.crosstab(index = data[col], columns = data[target])
            test_stat, p_value, _, _ = st.chi2_contingency(observed)
        else:
            test_stat, p_value = st.ttest_ind(a=counts[0],b=counts[1],equal_var=False)

        test_stats[col] = test_stat
        p_vals[col] = p_value

In [10]:
results = pd.DataFrame({'Feature' : list(p_vals.keys()),
                        'CV' : list(test_stats.values()),
                        'p value' : list(p_vals.values()),
                        'Significant' : ['S' if p_val < alpha else 'X' for p_val in p_vals.values()]
                        })

results_xl_path = "feature_tests.csv"

results.to_csv(results_xl_path, index = False)